### Przygotowanie danych

Naszym pierwszym zadaniem jest przygotowanie wsadu dla modeli uczenia maszynowego. Zrobimy to przy wykorzystaniu `numpy`, `pandas` i `rasterio`, generując szybko ramkę danych z kolumnami `NDVI`, `NDWI`, `SCI` i kolumną `y`, która reprezentować będzie dane binarne: **to, czy dany piksel jest terenem zabudowanym, czy nie jest**.

In [31]:
import numpy as np
import pandas as pd
import rasterio as rio

from keras.models import Sequential
from keras.layers import Dense

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
import matplotlib.pyplot as plt

from bands.load import LoadBandsFromTif

ImportError: cannot import name '_imaging' from 'PIL' (/usr/lib/python3/dist-packages/PIL/__init__.py)

In [3]:
BINARY_DATA = './results/clc_results/binarized_clc.tif'
NDVI_DATA = './results/ndvi.tif'
NDWI_DATA = './results/ndwi.tif'
SCI_DATA = './results/sci.tif'

Teraz ułożymy wartości do ramki danych, najpierw wczytamy wszystkie dane:

In [5]:
bin_band = LoadBandsFromTif(BINARY_DATA).get_band()
ndv_band = LoadBandsFromTif(NDVI_DATA).get_band()
ndw_band = LoadBandsFromTif(NDWI_DATA).get_band()
sci_band = LoadBandsFromTif(SCI_DATA).get_band()

Teraz utworzymy ramkę danych... najpierw wszystkie wartości:

In [6]:
arr = np.empty(shape=(len(bin_band.flatten()), 4))
arr.shape

(4445532, 4)

In [7]:
arr[:, 0] = bin_band.flatten()
arr[:, 1] = ndv_band.flatten()
arr[:, 2] = ndw_band.flatten()
arr[:, 3] = sci_band.flatten()

In [8]:
columns = ['y', 'ndvi', 'ndwi', 'sci']
df = pd.DataFrame(data=arr, columns=columns)
df.head()

,y,ndvi,ndwi,sci
0,0.0,0.861824,-0.827907,0.827907
1,0.0,0.857498,-0.818329,0.818329
2,0.0,0.857490,-0.826433,0.826433
3,0.0,0.868816,-0.825994,0.825994
4,0.0,0.865950,-0.835684,0.835684


Wczytamy podstawowe informacje o danych:

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4445532 entries, 0 to 4445531
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   y       float64
 1   ndvi    float64
 2   ndwi    float64
 3   sci     float64
dtypes: float64(4)
memory usage: 135.7 MB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
y,4445532.0,0.586137,0.492525,0.000000,0.000000,1.000000,1.000000,1.000000
ndvi,4445532.0,0.519922,0.254621,-0.740552,0.307663,0.571663,0.751201,1.000000
ndwi,4445532.0,-0.557063,0.207317,-1.000000,-0.727110,-0.611056,-0.411924,0.715823
sci,4445532.0,0.557063,0.207317,-0.715823,0.411924,0.611056,0.727110,1.000000


Dane wyglądają na prawidłowe.
Zapiszemy dataframe do pliku csv:

In [9]:
df.to_csv('./results/prepared_frame.csv')

Ponieważ powyższy plik csv jest bardzo duży nie dodałam go do repozytorium, dostępny jest pod linkiem: https://drive.google.com/file/d/19j4ZU3wLrPNlQbX7DQIPrINN6MxAnUC9/view?usp=share_link

### Transformacja danych wejściowych i podział na zbiór treningowy i testowy.

W tym kroku przygotowujemy dane jako wsad do modeli uczenia maszynowego. Jako że wskaźniki są już w przedziale -1 do 1, możemy tylko podzielić dane na zbiór treningowy i testowy. Wykorzystamy do tego możliwości, jakie daje pakiet `scikit-learn`.

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop('y', axis=1),
    df['y'],
    random_state=14)

In [12]:
print(x_train.describe().T)
print("------------------------------------------------------------------------------")
print(x_test.describe().T)

          count      mean       std       min       25%       50%       75%  \
ndvi  3334149.0  0.519918  0.254616 -0.740552  0.307630  0.571672  0.751152   
ndwi  3334149.0 -0.557048  0.207348 -1.000000 -0.727063 -0.611058 -0.411840   
sci   3334149.0  0.557048  0.207348 -0.715823  0.411840  0.611058  0.727063   

           max  
ndvi  1.000000  
ndwi  0.715823  
sci   1.000000  
------------------------------------------------------------------------------
          count      mean       std       min       25%       50%       75%  \
ndvi  1111383.0  0.519933  0.254637 -0.667292  0.307802  0.571625  0.751345   
ndwi  1111383.0 -0.557110  0.207225 -1.000000 -0.727244 -0.611053 -0.412178   
sci   1111383.0  0.557110  0.207225 -0.626313  0.412178  0.611053  0.727244   

           max  
ndvi  1.000000  
ndwi  0.626313  
sci   1.000000  


In [13]:
print("Zbiór y trening liczy", y_train.count(), "elementów.")
print("Zbiór y testowy liczy", y_test.count(), "elementów.")

Zbiór y trening liczy 3334149 elementów.
Zbiór y testowy liczy 1111383 elementów.


### Trening różnych klasyfikatorów dostępnych w pakiecie scikit-learn.

Najpierw przeprowadzimy trening różnych klasyfikatorów dostępnych w pakiecie scikit-learn. Spośród nich wybierzemy:

> I. Drzewa decyzyjne,

> II. Lasy losowe,

> III. Naiwny Bayesowski klasyfikator,

> IV. Ada Boost,

> V. prostą sieć neuronową.


In [15]:
# Drzewa decyzyjne

decision_tree_classifier = DecisionTreeClassifier(criterion='gini', 
                                                  max_depth=10, 
                                                  min_samples_split=3, 
                                                  random_state=14)

decision_tree_classifier.fit(x_train, y_train)

# Predykcje

decision_tree_predictions = decision_tree_classifier.predict(x_test)

In [17]:
# Lasy losowe

random_forest_classifier = RandomForestClassifier(n_estimators=20,
                                                  max_depth=10,
                                                  n_jobs=-1)
random_forest_classifier.fit(x_train, y_train)
predictions_random_forest = random_forest_classifier.predict(x_test)

In [18]:
# Naive Bayes

nb_classifier = GaussianNB()
nb_classifier.fit(x_train, y_train)
predictions_bayes = nb_classifier.predict(x_test)

In [19]:
# Ada Boost

adaboost_classifier = AdaBoostClassifier(n_estimators=20)
adaboost_classifier.fit(x_train, y_train)
predictions_ada = adaboost_classifier.predict(x_test)

In [20]:
# Sieć neuronowa

nn_classifier = MLPClassifier(hidden_layer_sizes=(16, 16, 8, 2),
                              max_iter=30,
                              n_iter_no_change=5)
nn_classifier.fit(x_train, y_train)
predictions_nn = nn_classifier.predict(x_test)

### Trening klasyfikatora opartego na uczeniu głębokim przy wykorzystaniu pakietu Keras.

W tym momencie moglibyśmy już przerwać nasz trening i przejść do analizy wyników. Jednak dla porównania zastosujemy również sieć neuronową opartą na pakiecie `keras` i module `Tensorflow`. Taką sieć buduje się nieco inaczej niż tę z pakietu `scikit-learn`.

Do trenowania sieci neuronowej opartej na uczeniu głębokim potrzebować będziemy również zbioru walidacyjnego, dlatego podzielimy jeszcze nasz zbiór treningowy na dwie części: 1/5 zbioru zostanie odłożona do walidacji trenowanego modelu.

In [ ]:
kmodel = Sequential()

In [ ]:
kmodel.add(Dense(16, activation='relu'))
kmodel.add(Dense(16, activation='relu'))
kmodel.add(Dense(8, activation='relu'))
kmodel.add(Dense(2, activation='relu'))
kmodel.add(Dense(1, activation='sigmoid'))

In [ ]:
kmodel.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
len(x_train)

In [ ]:
val_size = int(0.2 * len(x_train))

In [ ]:
x_val = x_train[:val_size]
x_train_kmodel = x_train[val_size:]

y_val = y_train[:val_size]
y_train_kmodel = y_train[val_size:]

In [ ]:
history = kmodel.fit(x_train_kmodel, y_train_kmodel,
                     epochs=10, batch_size=2048,
                     validation_data=(x_val, y_val))

Po wytrenowaniu modelu sprawdźmy stratę trenowania i walidacji oraz dokładność trenowania i walidacji, by przekonać się, czy nie wystąpiło zjawisko przewymiarowania.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Strata trenowania')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji')

plt.xlabel('Epoki')
plt.ylabel('Strata')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, acc, 'ro', label='Dokładność trenowania')
plt.plot(epochs, val_acc, 'r', label='Dokładność walidacji')

plt.xlabel('Epoki')
plt.ylabel('Dokładność')
plt.legend()
plt.show()

W tym wypadku dane nie są przewymiarowe, więc możemy dokonać predykcji.

In [ ]:
predicted_keras = kmodel.predict(x_test, batch_size=2048)
predicted_keras[predicted_keras > 0.5] = 1
predicted_keras[predicted_keras <= 0.5] = 0

### Zestawienie wyników poszczególnych modeli.

Jak sprawują się poszczególne modele? Możemy to zbadać wykorzystując w tym celu macierze błędów i inne metryki opracowane dla klasyfikatorów. Funkcje, które je obliczają, dostępne są w pakiecie `scikit-learn`.

In [25]:
def show_predicted_output(predictions, y_t):
    
    output = accuracy_score(y_t, predictions)
    confusion = confusion_matrix(y_t, predictions, normalize='true')
    print(confusion_matrix)
    
    sns.heatmap(confusion, annot=True, cmap='winter')

    print('Accuracy score is:', output)
    p_r_f1 = precision_recall_fscore_support(y_t, predictions, average='macro')
    print('Precision: {}, Recall: {}, F1 score: {}'.format(p_r_f1[0], p_r_f1[1], p_r_f1[2]))

In [32]:
# Drzewa decyzyjne

show_predicted_output(decision_tree_predictions, y_test)

<function confusion_matrix at 0x7fa9b7b6d2d0>


NameError: name 'sns' is not defined

In [ ]:
# Lasy Losowe

show_predicted_output(predictions_random_forest, y_test)

In [ ]:
# Bayes

show_predicted_output(predictions_bayes, y_test)

In [ ]:
# ADA

show_predicted_output(predictions_ada, y_test)

In [ ]:
# Sieci Neuronowe = scikit

show_predicted_output(predictions_nn, y_test)

In [ ]:
# Keras

show_predicted_output(predicted_keras, y_test)